# Exploration of WildFires Colombia

In [51]:
import pandas as pd
import geopandas as gpd
import glob
from unidecode import unidecode
from shapely.geometry import Point
import numpy as np
pd.set_option('future.no_silent_downcasting', True)
import plotly.express as px

## Reading and Consolidating Data

### Wildfires Data

In [2]:
df_wildfires=pd.read_parquet("../../Data/col_2000-2024.parquet")
print(df_wildfires.shape)
print(df_wildfires.dtypes)
df_wildfires.head(3)

(953822, 15)
latitude             float64
longitude            float64
brightness           float64
scan                 float64
track                float64
acq_date      datetime64[ns]
acq_time              object
satellite           category
instrument          category
confidence             int64
version               object
bright_t31           float64
frp                  float64
daynight            category
type                 float64
dtype: object


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,11.0912,-72.6625,306.5,4.5,2.0,2000-11-01,14:58:00,Terra,MODIS,26,6.2,277.3,38.5,D,2.0
1,11.0910,-72.6702,307.0,4.5,2.0,2000-11-01,14:58:00,Terra,MODIS,31,6.2,277.5,41.9,D,2.0
2,5.3136,-68.8234,316.9,1.1,1.1,2000-11-02,03:24:00,Terra,MODIS,94,6.2,278.9,19.6,N,0.0


In [3]:
df_wildfires=df_wildfires[["latitude", "longitude", "brightness", "acq_date", "acq_time", "confidence", "frp", "daynight", "type"]]
df_wildfires.head(3)

,latitude,longitude,brightness,acq_date,acq_time,confidence,frp,daynight,type
0,11.0912,-72.6625,306.5,2000-11-01,14:58:00,26,38.5,D,2.0
1,11.0910,-72.6702,307.0,2000-11-01,14:58:00,31,41.9,D,2.0
2,5.3136,-68.8234,316.9,2000-11-02,03:24:00,94,19.6,N,0.0


In [4]:
df_wildfires_0=df_wildfires[(df_wildfires["type"]==0.0)].reset_index(drop=True)
df_fires=df_wildfires_0.groupby(["latitude","longitude","acq_date"])[["brightness", "frp"]].max().reset_index()
df_fires

,latitude,longitude,acq_date,brightness,frp
0,-53.2230,-68.7157,2023-01-09,306.3,9.3
1,-53.2202,-68.7194,2023-01-04,303.8,5.9
2,-53.1107,-68.7938,2023-01-03,301.1,2.6
3,-52.8783,-68.8534,2023-01-18,313.6,9.8
4,-51.6627,-71.7810,2023-01-16,316.4,17.3
...,...,...,...,...,...
906153,58.4882,-119.9228,2023-01-20,319.2,26.3
906154,58.5065,-119.6110,2023-01-20,321.5,28.4
906155,58.5096,-119.5933,2023-01-20,308.8,16.3
906156,58.5153,-119.6168,2023-01-20,326.8,34.6


### Parques y Parámos

In [5]:
nat_parks = gpd.read_file('../../Data/national_parks_shapefile/runap2Polygon.shp')
print(nat_parks.shape)
print(nat_parks.dtypes)
nat_parks.head(1)

(1673, 21)
nombre          object
categoria       object
territoria      object
resolucion      object
hectareas_     float64
escala          object
organizaci      object
fecha_act       object
url             object
wkid            object
centroid_x     float64
centroid_y     float64
fecha_regi      object
validado        object
hectareas0     float64
perimetro_      object
administra      object
id_pnn           int64
record_id        int64
app_id           int64
geometry      geometry
dtype: object


,nombre,categoria,territoria,resolucion,hectareas_,escala,organizaci,fecha_act,url,wkid,...,centroid_y,fecha_regi,validado,hectareas0,perimetro_,administra,id_pnn,record_id,app_id,geometry
0,VillaLuz,Reserva Natural de la Sociedad Civil,None,0142,18.0,None,Parques Nacionales Naturales de Colombia,2011-07-26,https://runap.parquesnacionales.gov.co/area-pr...,3116,...,4.558803,2011-07-26,Si,19.452341,None,PNNC,334,153,1,"POLYGON ((-73.83242 4.55865, -73.83254 4.55860..."


In [6]:
df_paramos = gpd.read_file('../../Data/paramos_shapefile/paramos.shp')
print(df_paramos.shape)
print(df_paramos.dtypes)
df_paramos.head(1)

(55, 14)
OBJECTID         int64
SectorCod       object
Codigo          object
ComplejoNo      object
DistritoNo      object
SectorNomb      object
ComplejoCo      object
DistritoCo      object
AreaHa         float64
ActoAdmini      object
ObjetoCodi      object
Shape__Are     float64
Shape__Len     float64
geometry      geometry
dtype: object


,OBJECTID,SectorCod,Codigo,ComplejoNo,DistritoNo,SectorNomb,ComplejoCo,DistritoCo,AreaHa,ActoAdmini,ObjetoCodi,Shape__Are,Shape__Len,geometry
0,49,CE,CE-CM-CHG,Páramo de Chingaza,Cundinamarca,Cordillera Oriental,CHG,CM,11287.849924,Resolución MADS 0710 del 26 de mayo de 2016,CE-CM-CHG-8,1.128785e+08,133632.661234,"POLYGON ((-73.47134 4.62136, -73.47158 4.62133..."


In [7]:
df_paramos=df_paramos[["ComplejoNo", "geometry"]]
df_paramos.rename(columns={"ComplejoNo":"nombre"}, inplace=True)
df_paramos["categoria"]= "Páramo"
df_paramos.crs = "EPSG:4326"
print(df_paramos.dtypes)
df_paramos.head(3)

nombre         object
geometry     geometry
categoria      object
dtype: object


,nombre,geometry,categoria
0,Páramo de Chingaza,"POLYGON ((-73.47134 4.62136, -73.47158 4.62133...",Páramo
1,Páramo de Chingaza,"POLYGON ((-73.94728 4.66754, -73.94728 4.66753...",Páramo
2,Páramo de Chingaza,"POLYGON ((-73.63226 4.71737, -73.63225 4.71737...",Páramo


In [8]:
def clean_names(name):
    name = name.strip()  # Remove leading and trailing whitespace
    name = unidecode(name)  # Remove accents and diacritics
    name = name.lower()  # Convert to lowercase
    name = name.replace('a3', 'o')
    name = name.replace('(c)', 'e')
    name = name.replace('+-', 'n')
    name = name.replace('antionquia', 'antioquia')
    name = name.replace('sinao', 'sinu')
    name = name.replace('raos', 'rios')
    name = name.replace('anica', 'unica')
    name = name.replace('!', '')
    name = name.replace('vaa', 'via')
    return name

In [121]:
nat_parks['categoria'] = nat_parks['categoria'].apply(clean_names)
nat_parks['nombre'] = nat_parks['nombre'].apply(clean_names)
nat_parks['organizaci'] = nat_parks['organizaci'].apply(clean_names)

nat_parks.head(3)

,nombre,categoria,territoria,resolucion,hectareas_,escala,organizaci,fecha_act,url,wkid,centroid_x,centroid_y,fecha_regi,validado,hectareas0,perimetro_,administra,id_pnn,record_id,app_id,geometry
0,villaluz,reserva natural de la sociedad civil,None,0142,18.0,None,parques nacionales naturales de colombia,2011-07-26,https://runap.parquesnacionales.gov.co/area-pr...,3116,-73.836040,4.558803,2011-07-26,Si,19.452341,None,PNNC,334,153,1,"POLYGON ((-73.83242 4.55865, -73.83254 4.55860..."
1,la marcada,distritos de conservacion de suelos,None,010,1874.0,None,corporacion autonoma regional de risaralda,2016-12-21,https://runap.parquesnacionales.gov.co/area-pr...,3115,-75.615349,4.829014,2016-12-21,Si,1872.636332,None,CARDER,743,708,1,"POLYGON ((-75.63042 4.85554, -75.63044 4.85549..."
2,cuenca del rao tame,reservas forestales protectoras nacionales,None,"0063,128",1900.0,None,ministerio de ambiente y desarrollo sostenible,None,https://runap.parquesnacionales.gov.co/area-pr...,3117,-71.928365,6.393404,None,Si,1647.516286,None,MinAmbiente,421,661,1,"POLYGON ((-71.97163 6.42752, -71.97136 6.42733..."


In [9]:
df_parques=nat_parks[["nombre","categoria", "geometry"]]
df_parques.crs = "EPSG:4326"
print(df_parques.dtypes)
df_parques.head(2)

nombre         object
categoria      object
geometry     geometry
dtype: object


,nombre,categoria,geometry
0,VillaLuz,Reserva Natural de la Sociedad Civil,"POLYGON ((-73.83242 4.55865, -73.83254 4.55860..."
1,La Marcada,Distritos de ConservaciÃ³n de Suelos,"POLYGON ((-75.63042 4.85554, -75.63044 4.85549..."


In [10]:
# Final Polygons df
df_polygonos=pd.concat([df_parques, df_paramos])
df_polygonos.head()

,nombre,categoria,geometry
0,VillaLuz,Reserva Natural de la Sociedad Civil,"POLYGON ((-73.83242 4.55865, -73.83254 4.55860..."
1,La Marcada,Distritos de ConservaciÃ³n de Suelos,"POLYGON ((-75.63042 4.85554, -75.63044 4.85549..."
2,Cuenca del RÃ­o Tame,Reservas Forestales Protectoras Nacionales,"POLYGON ((-71.97163 6.42752, -71.97136 6.42733..."
3,El Arrullo,Reserva Natural de la Sociedad Civil,"POLYGON ((-75.98806 1.30587, -75.98626 1.30551..."
4,The Peak Regional Park,Parques Naturales Regionales,"POLYGON ((-81.37301 13.35062, -81.37299 13.350..."


### Joining Data from WildFires with Paramos Polygons

In [27]:
# Adding tag if fire is in National Park

gdf_wildfires = gpd.GeoDataFrame(df_fires, geometry=gpd.points_from_xy(df_fires.longitude, df_fires.latitude))
#gdf_wildfires.rename(columns={"geomet"})
# Set the CRS for gdf_wildfires to WGS84 (EPSG:4326) - This should match your df_paramos CRS
gdf_wildfires.crs = "EPSG:4326"
df_parques.crs = "EPSG:4326"
df_paramos.crs = "EPSG:4326"

# Add flag to fires in national parks
joined_df = gpd.sjoin(gdf_wildfires, df_parques, how="left", predicate='within')
joined_df.loc[joined_df.nombre.isna(), "in_park"]=False
joined_df["in_park"]=joined_df["in_park"].fillna(True)
joined_df["nombre"]=joined_df["nombre"].fillna("No Aplica")
joined_df["categoria"]=joined_df["categoria"].fillna("No Aplica")
#
joined_df=joined_df[['latitude', 'longitude', 'acq_date', 'brightness', 'frp', "nombre", "categoria", "in_park"]]

gdf_wildfires = gpd.GeoDataFrame(joined_df, geometry=gpd.points_from_xy(joined_df.longitude, joined_df.latitude))
gdf_wildfires.crs = "EPSG:4326"

# Add flag to fires in paramos
joined_df = gpd.sjoin(gdf_wildfires, df_paramos, how="left", predicate='within')
joined_df.loc[joined_df.nombre_right.isna(), "in_paramo"]=False
joined_df["in_paramo"]=joined_df["in_paramo"].fillna(True)
joined_df.rename(columns={"acq_date":"date", "nombre_left":"nombre_parque",
        "categoria_left":"categoria", "nombre_right": "nombre_paramo"}, inplace=True)
joined_df["nombre_paramo"]=joined_df["nombre_paramo"].fillna("No Aplica")

joined_df=joined_df[['latitude', 'longitude', 'date', 'brightness', 'frp', "categoria",
         "nombre_parque", "nombre_paramo", "in_park", "in_paramo"]]

joined_df.head()

,latitude,longitude,date,brightness,frp,categoria,nombre_parque,nombre_paramo,in_park,in_paramo
0,-53.2230,-68.7157,2023-01-09,306.3,9.3,No Aplica,No Aplica,No Aplica,False,False
1,-53.2202,-68.7194,2023-01-04,303.8,5.9,No Aplica,No Aplica,No Aplica,False,False
2,-53.1107,-68.7938,2023-01-03,301.1,2.6,No Aplica,No Aplica,No Aplica,False,False
3,-52.8783,-68.8534,2023-01-18,313.6,9.8,No Aplica,No Aplica,No Aplica,False,False
4,-51.6627,-71.7810,2023-01-16,316.4,17.3,No Aplica,No Aplica,No Aplica,False,False


In [80]:
joined_df.groupby(["nombre_paramo"])[["in_park"]].count().reset_index().sort_values(by="in_park",ascending=False)

,nombre_paramo,in_park
4,No Aplica,905732
1,Cruz Verde - Sumapaz,465
6,Rabanal y río Bogotá,27
5,Páramo de Chingaza,24
2,Guerrero,12
3,Iguaque - Merchán,6
0,Altiplano Cundiboyacense,4


In [77]:
def fires_evolution_in_parks(df, min_date: str, max_date: str, nombre_parque: str, in_park: bool):

    df_temp=df[(df["date"]>=min_date) & (df["date"]<=max_date) & (df["in_park"]==in_park) & (df["nombre_parque"]==nombre_parque)]
    df_temp= df_temp[["date", "brightness", "categoria", "nombre_parque", "in_park"]]
    df_temp=df_temp.groupby(["date", "categoria", "nombre_parque"])["brightness"].max().reset_index()

    df_temp.set_index('date', inplace=True)

    # Resample data to get mean values per month and round to 1 decimal
    df_temp = df_temp.resample('W').max().round(1)

    # Reset index to have 'date' as a regular column
    df_temp.reset_index(inplace=True)
    df_temp['id'] = range(len(df_temp))

    # Check if df_temp is empty
    if df_temp.empty:
        max_dates = max_date  # Use the user-specified max_date as a fallback
    else:
        max_dates = df_temp["date"].max().strftime("%Y-%m-%d")

    frames = []
    for i in range(len(df_temp)):
        frame = df_temp[df_temp['id'] <= i].copy()
        frame['frame'] = i
        frames.append(frame)
        
    # Check if frames is not empty before concatenation
    if frames:
        animated_df = pd.concat(frames)
    else:
        # Handling the case where frames is empty
        # Create an empty DataFrame with necessary columns for plotly to avoid error
        animated_df = pd.DataFrame(columns=['date', "brightness", 'id', 'frame'])


    if animated_df.empty:
        fig = px.scatter(title=f'Sin datos disponibles para {nombre_parque.capitalize()} entre {min_date} y {max_dates}')
    else:
        fig = px.scatter(animated_df, x='date', y="brightness", 
                         title=f'Evolución de incendios en {nombre_parque.capitalize()} entre {min_date} y {max_dates}',
                         labels= {'date': 'Fecha'},
                         animation_frame='frame',
                         color="brightness",
                         color_continuous_scale=px.colors.sequential.YlOrRd,
                         opacity=0.7,
                         template="plotly_dark",
                        
        )#template="none")

        
        # Set all points to the same size using update_traces
        fig.update_traces(marker=dict(size=20))

        # Set animation duration
        fig.update_layout(
            updatemenus=[dict(type='buttons', 
                                showactive=False, 
                                buttons=[dict(label='Play',
                                            method='animate',
                                            args=[None, dict(frame=dict(duration=10, redraw=True), fromcurrent=True)]),
                                        dict(label='Pause',
                                            method='animate',
                                            args=[[None], dict(frame=dict(duration=0, redraw=True), mode='immediate', transition=dict(duration=0))])])],
            #height=500,  # Set the height of the plot here. Adjust the value as needed.
            title=dict(text=f'Evolución de incendios en {nombre_parque.capitalize()} entre {min_date} y {max_dates}',
                       x=0.5,  # Center the title
                       xanchor='center',  # Use 'center' to center
                       font=dict(size=20)  # Adjust the font size here
                      )
                                                      )
        fig.update_xaxes(type='date', tickformat='%Y-%m', title_text='Fecha', tickangle=45)
        fig.update_xaxes(tickmode='linear', tick0=animated_df['date'].min(), dtick='M1')
    return fig.show()
    

In [78]:
fires_evolution_in_parks(joined_df, "2020-01-01", "2023-12-31", "Cinaruco", True)

In [91]:
def fires_evolution_in_paramos(df, min_date: str, max_date: str, nombre_paramo: str, in_paramo: bool):

    df_temp=df[(df["date"]>=min_date) & (df["date"]<=max_date) & (df["in_paramo"]==in_paramo) & (df["nombre_paramo"]==nombre_paramo)]
    df_temp= df_temp[["date", "brightness", "categoria", "nombre_paramo", "in_paramo"]]
    df_temp=df_temp.groupby(["date", "categoria", "nombre_paramo"])["brightness"].max().reset_index()

    df_temp.set_index('date', inplace=True)

    # Resample data to get mean values per month and round to 1 decimal
    df_temp = df_temp.resample('W').max().round(1)

    # Reset index to have 'date' as a regular column
    df_temp.reset_index(inplace=True)
    df_temp['id'] = range(len(df_temp))

    # Check if df_temp is empty
    if df_temp.empty:
        max_dates = max_date  # Use the user-specified max_date as a fallback
    else:
        max_dates = df_temp["date"].max().strftime("%Y-%m-%d")

    frames = []
    for i in range(len(df_temp)):
        frame = df_temp[df_temp['id'] <= i].copy()
        frame['frame'] = i
        frames.append(frame)
        
    # Check if frames is not empty before concatenation
    if frames:
        animated_df = pd.concat(frames)
    else:
        # Handling the case where frames is empty
        # Create an empty DataFrame with necessary columns for plotly to avoid error
        animated_df = pd.DataFrame(columns=['date', "brightness", 'id', 'frame'])


    if animated_df.empty:
        fig = px.scatter(title=f'Sin datos disponibles para {nombre_paramo.capitalize()} entre {min_date} y {max_dates}')
    else:
        fig = px.scatter(animated_df, x='date', y="brightness", 
                         title=f'Evolución de incendios en {nombre_paramo.capitalize()} entre {min_date} y {max_dates}',
                         labels= {'date': 'Fecha'},
                         animation_frame='frame',
                         color="brightness",
                         color_continuous_scale=px.colors.sequential.YlOrRd,
                         opacity=0.7,
                         template="plotly_dark",
                        
        )#template="none")

        
        # Set all points to the same size using update_traces
        fig.update_traces(marker=dict(size=20))

        # Set animation duration
        fig.update_layout(
            updatemenus=[dict(type='buttons', 
                                showactive=False, 
                                buttons=[dict(label='Play',
                                            method='animate',
                                            args=[None, dict(frame=dict(duration=10, redraw=True), fromcurrent=True)]),
                                        dict(label='Pause',
                                            method='animate',
                                            args=[[None], dict(frame=dict(duration=0, redraw=True), mode='immediate', transition=dict(duration=0))])])],
            #height=500,  # Set the height of the plot here. Adjust the value as needed.
            title=dict(text=f'Evolución de incendios en {nombre_paramo.capitalize()} entre {min_date} y {max_dates}',
                       x=0.5,  # Center the title
                       xanchor='center',  # Use 'center' to center
                       font=dict(size=20)  # Adjust the font size here
                      )
                                                      )
        fig.update_xaxes(type='date', tickformat='%Y-%m-%d', title_text='Fecha', tickangle=45)
        fig.update_xaxes(tickmode='linear', tick0=animated_df['date'].min(), dtick='M1')
    return fig.show()

In [92]:
fires_evolution_in_paramos(joined_df, "2020-01-01", "2023-12-31", "Cruz Verde - Sumapaz", True)

In [ ]:
def monthly_evolution_of_temperature_per_municipio(df, departamento, municipio, min_date, max_date, variable):
    labels_vars = {
        "temp_max": ["Temperatura máxima", "°C", "promedio"],
        "temp_avg": ["Temperatura promedio", "°C", "promedio"],
        "temp_min": ["Temperatura mínima", "°C", "promedio"],
        "precipitacion_total": ["Precipitación total","mm", "suma"]
    }

    df_temp = select_municipio(df, municipio)

    df_temp.set_index('date', inplace=True)

    # Resample data to get mean values per month and round to 1 decimal
    df_temp = df_temp.resample('M').mean().round(1)

    # Reset index to have 'date' as a regular column
    df_temp.reset_index(inplace=True)

    if variable=="precipitacion_total":
        df_temp = df_temp[(df_temp[variable] >= 0) & (df_temp["date"] >= pd.to_datetime(min_date)) & (df_temp["date"] <= pd.to_datetime(max_date))]
        df_temp['id'] = range(len(df_temp))
    else:    
        df_temp = df_temp[(df_temp[variable] >= 0) & (df_temp[variable] <= 45) & (df_temp["date"] >= pd.to_datetime(min_date)) & (df_temp["date"] <= pd.to_datetime(max_date))]
        df_temp['id'] = range(len(df_temp))

     # Check if df_temp is empty
    if df_temp.empty:
        max_dates = max_date  # Use the user-specified max_date as a fallback
    else:
        max_dates = df_temp["date"].max().strftime("%Y-%m-%d")

    frames = []
    for i in range(len(df_temp)):
        frame = df_temp[df_temp['id'] <= i].copy()
        frame['frame'] = i
        frames.append(frame)
        
    # Check if frames is not empty before concatenation
    if frames:
        animated_df = pd.concat(frames)
    else:
        # Handling the case where frames is empty
        # Create an empty DataFrame with necessary columns for plotly to avoid error
        animated_df = pd.DataFrame(columns=['date', variable, 'id', 'frame'])


    if animated_df.empty:
        fig = px.scatter(title=f'Sin datos disponibles para {municipio.capitalize()}, {departamento.capitalize()} entre {min_date} y {max_dates}')
    else:
        fig = px.scatter(animated_df, x='date', y=variable, 
                         title=f'Evolución de {labels_vars[variable][2]} de {labels_vars[variable][0]}({labels_vars[variable][1]}) Mensual de {municipio.capitalize()}, {departamento.capitalize()} entre {min_date} y {max_dates}',
                         labels= {variable: labels_vars[variable][1], 'date': 'Fecha'},
                         animation_frame='frame',
                         color=variable,
                         color_continuous_scale=px.colors.diverging.Portland,
                         opacity=0.7,
        )#template="none")

        
        # Set all points to the same size using update_traces
        fig.update_traces(marker=dict(size=30))

        # Set animation duration
        fig.update_layout(
            updatemenus=[dict(type='buttons', 
                                showactive=False, 
                                buttons=[dict(label='Play',
                                            method='animate',
                                            args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True)]),
                                        dict(label='Pause',
                                            method='animate',
                                            args=[[None], dict(frame=dict(duration=0, redraw=True), mode='immediate', transition=dict(duration=0))])])],
            #height=500,  # Set the height of the plot here. Adjust the value as needed.
            title=dict(text=f'Evolución de {labels_vars[variable][2]} de {labels_vars[variable][0]}({labels_vars[variable][1]}) Mensual de {municipio.capitalize()}, {departamento.capitalize()} entre {min_date} y {max_dates}',
                       x=0.5,  # Center the title
                       xanchor='center',  # Use 'center' to center
                       font=dict(size=20)  # Adjust the font size here
                      )
                                                      )
        fig.update_xaxes(type='date', tickformat='%Y-%m', title_text='Fecha', tickangle=45)
        fig.update_xaxes(tickmode='linear', tick0=animated_df['date'].min(), dtick='M3')
    return fig

In [49]:
import folium
import geopandas as gpd

def create_paramos_map(df_paramos):
    """
    Create a Folium map with polygons from a GeoDataFrame.
    
    Parameters:
    - df_paramos: GeoDataFrame containing the polygons and data of páramos.
    
    Returns:
    - A Folium Map object with the polygons added.
    """
    # Initialize the map centered around Colombia
    m = folium.Map(location=[4.5709, -74.2973], zoom_start=6)
    
    def add_polygon_to_map(row, folium_map):
        """
        Add a polygon to the provided folium map based on GeoDataFrame row.
        """
        # Simplify geometry to reduce complexity and ease rendering
        simplified_geo = row['geometry'].simplify(0.005, preserve_topology=False)
        # Create a GeoJson object from the geometry
        geo_j = folium.GeoJson(data=simplified_geo,
                               style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'weight': 2})
        # Add a popup that shows the 'ComplejoNo' (or any other property you're interested in)
        folium.Popup(row['ComplejoNo']).add_to(geo_j)
        geo_j.add_to(folium_map)
    
    # Apply the function to each row of the GeoDataFrame
    df_paramos.apply(add_polygon_to_map, folium_map=m, axis=1)
    
    return m

# Assuming you have a GeoDataFrame called df_paramos ready
# map_object = create_paramos_map(df_paramos)
# Display the map in a Jupyter notebook using:
# map_object



In [50]:
map_object = create_paramos_map(df_paramos)
map_object


In [52]:
map_object = create_paramos_map(nat_parks)
map_object


KeyError: 'ComplejoNo'